In [1]:
from pylab import*
import pandas as pd
import torch
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/restaurants_tripadvisor_reviews.csv'
path2 = '/content/drive/My Drive/restaurants_google_with_reviews.csv'
path3 = '/content/drive/My Drive/restaurants_paris_with_url.csv'
path4='/content/drive/My Drive/restaurants_tripadvisor_paris_final.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install googletrans==4.0.0-rc1

In [13]:
from googletrans import Translator
from langdetect import detect

def trans_en(text):
    if type(text) != str:
        return None
    if len(text) < 2:
        return text
    try:
        detected_lang = detect(text)
    except:
        return "not translated: " + text

    if detected_lang == 'en':
        return text
    else:
        try:
            translator = Translator()
            translated_text = translator.translate(text, src=detected_lang, dest='en').text
            return translated_text
        except Exception as e:
            return f"Translation error: {str(e)}"


In [5]:
review_trip=pd.read_csv(path)
review_trip

,id,text,title,rating,trip_type,location_id
0,987124639,I actually came every single time I’ve been in...,Asking for a tip,4,Solo travel,17227776
1,972571018,My daughter and i came here for a late lunch o...,"Relaxing Lunch, great spot !",5,Family,17227776
2,948361293,Had a chocolate mousse with a couple of drinks...,Perfect cocktail spot,5,Couples,17227776
3,924882190,"Loved this resraurant and it's location, food ...",Delicious French food,5,Couples,17227776
4,903705071,We had drinks several days in a row. Friendly ...,Nice place for drinks.,5,Family,17227776
...,...,...,...,...,...,...
4984,988863188,I don't know who they hired to post those revi...,"Cheap, yes. Mediocre, yes.",2,Solo travel,25165823
4985,988027209,I had a fantastic experience at Le P'tit Piano...,Great Dining Experience at Le P'tit Piano,4,Friends getaway,25165823
4986,976668123,"Great good, fast serving and nice personnel. T...",Great place to eat,5,Family,25165823
4987,955199173,"Good service, large portions and tasty food. ...","Friendly service, big portions and tasty food",5,Couples,25165823


In [6]:
review_trip['t_t']=review_trip['title'].str.lower()+' '+review_trip['text'].str.lower()
review_trip['t_t']=review_trip['t_t'].str.replace('\n',' ')
review_trip['t_t']=review_trip['t_t'].str.replace('[^\w\s]','')
rt=review_trip.groupby('location_id')['t_t'].sum()
review_trip['t_t_sum']=review_trip['location_id'].map(rt)
rewt=review_trip.copy()
rewt.drop(columns=['title','text','id','trip_type','rating','t_t'],inplace=True)
rewt=rewt.drop_duplicates()
rewt

,location_id,t_t_sum
0,17227776,asking for a tip i actually came every single ...
5,3727361,pret a manger charles de gaule aeroport to be ...
10,1056772,"relaxing spt, good food, good service a relaxi..."
15,11673606,what happened? quality has nose dived big fer...
20,5799949,great dinner found this place the second night...
...,...,...
4964,7602128,go !!! an authentic taste of asia every time! ...
4969,17727441,a very good find near the jardin de plantes a ...
4974,1007574,great value fantastic value! my 13 year old an...
4979,7282685,stay away! the food and service were terrible....


In [7]:
restaurant_trip=pd.read_csv(path4)
restaurant_trip=restaurant_trip[restaurant_trip['num_reviews']>0]
print(restaurant_trip.shape)
restaurant_trip.isnull().sum()/len(restaurant_trip)

(1676, 9)


,0
location_id,0.000000
name,0.000000
description,0.654535
adress_string,0.000000
price_level,0.173628
rating,0.001193
num_reviews,0.000000
review_rating_count,0.000000
cuisine,0.106205


In [8]:
trip_df=pd.merge(rewt,restaurant_trip,on='location_id')
trip_df['description']=trip_df['description'].apply(lambda x: str(x).lower())
trip_df['description']=trip_df['description'].str.replace('\n',' ')
trip_df['description']=trip_df['description'].str.replace('nan',' ')
trip_df.dropna(axis=0,inplace=True)
trip_df.drop_duplicates(inplace=True)
from tqdm import tqdm
tqdm.pandas()
trip_df['cuisine']=trip_df['cuisine'].progress_apply(lambda x: trans_en(x))
trip_df['description']=trip_df['description'].progress_apply(lambda x: trans_en(x))
price_dict={'€':20,'€€ - €€€':50,'€€€€':100}
trip_df['price_level']=trip_df['price_level'].map(price_dict)
trip_df['t_t_des']=trip_df['description']+' '+'['+trip_df['t_t_sum']+']'+' '+trip_df['cuisine']
trip_df.rename(columns={'price_level':'price'},inplace=True)
trip_df['combined_desc_reviews_type']=trip_df['t_t_des']
trip_df.rename(columns={'adress_string':'address'},inplace=True)
trip_df.rename(columns={'cuisine':'restaurant_type'},inplace=True)
trip_df

100%|██████████| 1133/1133 [06:20<00:00,  2.98it/s]


'The Midi canteen or the aperitif of the start evening.coffee, cold drinks, cocktails and gourmet plates.It is every day from 8 a.m. to 2 a.m.!'

In [16]:
trip_df

,location_id,t_t_sum,name,description,adress_string,price,rating,num_reviews,review_rating_count,cuisine,t_t_des
0,17227776,asking for a tip i actually came every single ...,Le Oken,,"19 rue Rambuteau, 75004 Paris France",50,4.0,130,"{'1': '14', '2': '14', '3': '11', '4': '39', '...","French, coffee, seafood and fish, European, di...",[asking for a tip i actually came every sing...
3,1056772,"relaxing spt, good food, good service a relaxi...",Le Tournesol cafe,The Midi canteen or the aperitif of the start ...,"9 rue de la Gaite, 75014 Paris France",50,2.5,323,"{'1': '114', '2': '38', '3': '61', '4': '75', ...","French, bar",The Midi canteen or the aperitif of the start ...
4,11673606,what happened? quality has nose dived big fer...,Big Fernand Paris Montparnasse,,"86 Boulevard du Montparnasse, 75014 Paris France",50,3.5,155,"{'1': '21', '2': '22', '3': '28', '4': '52', '...","French, American, fast food",[what happened? quality has nose dived big ...
5,5799949,great dinner found this place the second night...,Come Prima,,"76 Rue Alexandre Dumas, 75020 Paris France",20,4.5,43,"{'1': '0', '2': '1', '3': '1', '4': '13', '5':...","Italienne, pizza",[great dinner found this place the second ni...
6,1638417,very standard and cold we visited here at the ...,Hippopotamus Steakhouse,Hippopotamus restaurant Paris Bercy village 12...,"27/29 Cour Saint-emilion Bercy Village, 75012 ...",50,3.5,243,"{'1': '48', '2': '30', '3': '32', '4': '60', '...","French, steakhouse",Hippopotamus restaurant Paris Bercy village 12...
...,...,...,...,...,...,...,...,...,...,...,...
1323,7602128,go !!! an authentic taste of asia every time! ...,4 Saisons,,"90 rue de Provence, 75009 Paris France",20,3.0,51,"{'1': '12', '2': '2', '3': '10', '4': '16', '5...","Asian, Vietnamese, Japanese, soups",[go !!! an authentic taste of asia every tim...
1324,17727441,a very good find near the jardin de plantes a ...,Les Petits Pois,"Bistronomic restaurant, fresh products and hom...","3 rue Linné, 75005 Paris France",50,4.5,106,"{'1': '3', '2': '3', '3': '4', '4': '9', '5': ...","French, European, healthy","Bistronomic restaurant, fresh products and hom..."
1325,1007574,great value fantastic value! my 13 year old an...,aki,,"16 Rue Saint Anne, 75001 Paris France",50,4.0,954,"{'1': '33', '2': '47', '3': '102', '4': '455',...","Japanese, Asian, healthy",[great value fantastic value! my 13 year old...
1326,7282685,stay away! the food and service were terrible....,Le Metro,,"37 boulevard de Belleville, 75011 Paris France",50,2.5,19,"{'1': '7', '2': '2', '3': '3', '4': '5', '5': ...",French,[stay away! the food and service were terrib...


In [10]:
trip_df.isnull().sum()/len(trip_df)

,0
location_id,0.0
t_t_sum,0.0
name,0.0
description,0.0
adress_string,0.0
price,0.0
rating,0.0
num_reviews,0.0
review_rating_count,0.0
cuisine,0.0


In [14]:
google_df=pd.read_csv(path2)
google_df.drop(columns=['google_maps_url'],inplace=True)
google_df.drop(columns=['reviews_url'],inplace=True)
google_df['description']=google_df['description'].apply(lambda x: str(x).lower())
google_df['description']=google_df['description'].str.replace('\n',' ')
google_df['description']=google_df['description'].str.replace('nan',' ')
google_df['reviews']=google_df['reviews'].progress_apply(lambda x: trans_en(x))
google_df.dropna(axis=0,inplace=True)
price_dict_g={'€30–40':35, '€20–30':25, '€20–40':30, '$$':30, '€40–50':45, '€10–20':15, '€10–30':20,'$$$':45, '€100+':150, nan:-1, '€30–50':40, '€50–60':55, '$$$$':60}
google_df['price']=google_df['price'].map(price_dict_g)
google_df=google_df[google_df['price']!=-1]
google_df['t_t_des']=google_df['description']+' '+google_df['reviews']+' '+google_df['restaurant_type	']
google_df['combined_desc_reviews_type']=google_df['t_t_des']
google_df

100%|██████████| 380/380 [06:36<00:00,  1.04s/it]


,name,rating,price,restaurant_type,description,address,reviews,t_t_des
0,L'Aubergeade,4.8,35,Restaurant,"typical, cozy bistro focusing on traditional f...","17 Rue Chaligny, 75012 Paris, France","[""A faultless for the hostel. We went there fo...","typical, cozy bistro focusing on traditional f..."
1,La Cantine de daumesnil - restaurant paris 12,4.3,25,Restaurant,,"119 Rue de Reuilly, 75012 Paris, France",['I recommend for lovers of West Indian dishes...,['I recommend for lovers of West Indian dish...
2,Le Picotin,4.5,30,French restaurant,"classic fare plated in a homey, relaxed settin...","35 Rue Sibuet, 75012 Paris, France",['Restaurant hidden in a calm alley. \ Nle ser...,"classic fare plated in a homey, relaxed settin..."
3,L'Alchimiste,4.7,25,French restaurant,,"181 Rue de Charenton, 75012 Paris, France","[""Very pleasant setting, top reception and ver...","[""Very pleasant setting, top reception and v..."
4,Ty Louis - Restaurant Breton Paris 12,4.8,30,Restaurant,,"240 Rue du Faubourg Saint-Antoine, 75012 Paris...","['And no, it is not because it is Breton that ...","['And no, it is not because it is Breton tha..."
...,...,...,...,...,...,...,...,...
375,Break | Restaurant Paris 17 | Terrasse | Brunc...,4.4,25,Restaurant,,"119 Bd Pereire, 75017 Paris, France",['Excellent brunch in a pop & tennis atmospher...,['Excellent brunch in a pop & tennis atmosph...
376,Restaurant Caïus,4.6,45,French restaurant,informal bistro serving wine & classic entrees...,"6 Rue d'Armaillé, 75017 Paris, France","['No complaints on the kitchen, beautiful pres...",informal bistro serving wine & classic entrees...
377,Restaurant Janine,4.7,30,Restaurant,,"90 Rue des Dames, 75017 Paris, France","['A first for me with the hare to the royal, a...","['A first for me with the hare to the royal,..."
378,Le P'tit Canon,4.4,30,French restaurant,,"36 Rue Legendre, 75017 Paris, France","[""During our visit to the little canon, we wer...","[""During our visit to the little canon, we w..."


In [15]:
google_df.isnull().sum()/len(google_df)

,0
name,0.0
rating,0.0
price,0.0
restaurant_type,0.0
description,0.0
address,0.0
reviews,0.0
t_t_des,0.0


In [21]:
trip_df.rename(columns={'adress':'address'},inplace=True)

In [22]:
df=pd.concat([trip_df[['name','rating','price','restaurant_type','address','combined_desc_reviews_type']],google_df[['name','rating','price','restaurant_type','address','combined_desc_reviews_type']]],axis=0)
df

,name,rating,price,restaurant_type,address,combined_desc_reviews_type
0,Le Oken,4.0,50,"French, coffee, seafood and fish, European, di...","19 rue Rambuteau, 75004 Paris France",[asking for a tip i actually came every sing...
3,Le Tournesol cafe,2.5,50,"French, bar","9 rue de la Gaite, 75014 Paris France",The Midi canteen or the aperitif of the start ...
4,Big Fernand Paris Montparnasse,3.5,50,"French, American, fast food","86 Boulevard du Montparnasse, 75014 Paris France",[what happened? quality has nose dived big ...
5,Come Prima,4.5,20,"Italienne, pizza","76 Rue Alexandre Dumas, 75020 Paris France",[great dinner found this place the second ni...
6,Hippopotamus Steakhouse,3.5,50,"French, steakhouse","27/29 Cour Saint-emilion Bercy Village, 75012 ...",Hippopotamus restaurant Paris Bercy village 12...
...,...,...,...,...,...,...
375,Break | Restaurant Paris 17 | Terrasse | Brunc...,4.4,25,Restaurant,"119 Bd Pereire, 75017 Paris, France",['Excellent brunch in a pop & tennis atmosph...
376,Restaurant Caïus,4.6,45,French restaurant,"6 Rue d'Armaillé, 75017 Paris, France",informal bistro serving wine & classic entrees...
377,Restaurant Janine,4.7,30,Restaurant,"90 Rue des Dames, 75017 Paris, France","['A first for me with the hare to the royal,..."
378,Le P'tit Canon,4.4,30,French restaurant,"36 Rue Legendre, 75017 Paris, France","[""During our visit to the little canon, we w..."


In [23]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# modele chargé
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
df['embeding']=df['combined_desc_reviews_type'].progress_apply(lambda x: model.encode(x).tolist())

100%|██████████| 1452/1452 [01:18<00:00, 18.61it/s]


In [25]:
df

,name,rating,price,restaurant_type,address,combined_desc_reviews_type,embeding
0,Le Oken,4.0,50,"French, coffee, seafood and fish, European, di...","19 rue Rambuteau, 75004 Paris France",[asking for a tip i actually came every sing...,"[0.0232643224298954, -0.020983874797821045, 0...."
3,Le Tournesol cafe,2.5,50,"French, bar","9 rue de la Gaite, 75014 Paris France",The Midi canteen or the aperitif of the start ...,"[-0.0033877294044941664, -0.006533175706863403..."
4,Big Fernand Paris Montparnasse,3.5,50,"French, American, fast food","86 Boulevard du Montparnasse, 75014 Paris France",[what happened? quality has nose dived big ...,"[0.055836353451013565, -0.020759504288434982, ..."
5,Come Prima,4.5,20,"Italienne, pizza","76 Rue Alexandre Dumas, 75020 Paris France",[great dinner found this place the second ni...,"[-0.008125054650008678, 0.023026341572403908, ..."
6,Hippopotamus Steakhouse,3.5,50,"French, steakhouse","27/29 Cour Saint-emilion Bercy Village, 75012 ...",Hippopotamus restaurant Paris Bercy village 12...,"[-0.019216522574424744, 0.05370445176959038, 0..."
...,...,...,...,...,...,...,...
375,Break | Restaurant Paris 17 | Terrasse | Brunc...,4.4,25,Restaurant,"119 Bd Pereire, 75017 Paris, France",['Excellent brunch in a pop & tennis atmosph...,"[-0.03475665673613548, 0.01653040014207363, 0...."
376,Restaurant Caïus,4.6,45,French restaurant,"6 Rue d'Armaillé, 75017 Paris, France",informal bistro serving wine & classic entrees...,"[-0.008625995367765427, 0.04631707817316055, 0..."
377,Restaurant Janine,4.7,30,Restaurant,"90 Rue des Dames, 75017 Paris, France","['A first for me with the hare to the royal,...","[-0.0160722304135561, 0.02327805571258068, 0.0..."
378,Le P'tit Canon,4.4,30,French restaurant,"36 Rue Legendre, 75017 Paris, France","[""During our visit to the little canon, we w...","[0.039266761392354965, 0.044852640479803085, -..."


In [26]:
df.to_csv('restaurants.csv',index=False)